In [1]:
import numpy as np
import pandas as pd
import re
import json
import requests
from xml.etree import ElementTree
import time
from dotenv import load_dotenv
import ast  # To safely evaluate string representation of a list
import bdikit as bdi
import os
from experiment_utils import *

In [2]:
pmid_doi_mapping_file = "exp_output/pmid_doi_mapping.json"
if os.path.exists(pmid_doi_mapping_file): 
    with open(pmid_doi_mapping_file, "r") as f:
        pmid_doi_mapping = json.load(f)  # read json file
else:
    pmid_doi_mapping = {}

In [3]:
pmid_pmcid_mapping_file = "exp_output/pmid_pmcid_mapping.json"
if os.path.exists(pmid_pmcid_mapping_file): 
    with open(pmid_pmcid_mapping_file, "r") as f:
        pmid_pmcid_mapping = json.load(f)  # read json file
else:
    pmid_pmcid_mapping = {}

In [4]:
doi_pmid_mapping_file = "exp_output/doi_pmid_mapping.json"
if os.path.exists(doi_pmid_mapping_file): 
    with open(doi_pmid_mapping_file, "r") as f:
        doi_pmid_mapping = json.load(f)  # read json file
else:
    doi_pmid_mapping = {}

In [5]:
doi_to_pmcid_mapping_file = "exp_output/doi_pmcid_mapping.json"
if os.path.exists(doi_to_pmcid_mapping_file):
    with open(doi_to_pmcid_mapping_file, "r") as f:
        doi_pmcid_mapping = json.load(f)  # read json file
else:
    doi_pmcid_mapping = {}

In [6]:
len(pmid_doi_mapping), len(pmid_pmcid_mapping), len(doi_pmid_mapping), len(doi_pmcid_mapping)

(144993, 144993, 125273, 125011)

## Extract the datasets data from ProteomeCentral and GEO

Start from the export file from ProteomeCentral: https://proteomecentral.proteomexchange.org/

In [7]:
export_df = pd.read_csv('exp_input/proteomexchange_search.tsv', sep='\t')
print(f"number of dataset ids in export dataframe from ProteomeCentral: {len(export_df)}")

number of dataset ids in export dataframe from ProteomeCentral: 40137


to get an idea of the most frequent values in df publication, for all the dataset ids from the source file

In [8]:
export_df['publication'].value_counts().head()

publication
Dataset with its publication pending                                                                 11719
no publication                                                                                        2550
<a href="http://www.ncbi.nlm.nih.gov/pubmed/35084980" target="_blank">Melani et al. (2022)</a>          56
<a href="http://www.ncbi.nlm.nih.gov/pubmed/28267743" target="_blank">Matsumoto et al. (2017)</a>       28
<a href="http://www.ncbi.nlm.nih.gov/pubmed/28071820" target="_blank">Kreutz et al. (2017)</a>          18
Name: count, dtype: int64

Clean the `publication` column by filtering out unwanted values like `Dataset with its publication pending`, `no publication`. Remove rows with unwanted values

In [9]:
filtered_df = export_df[~export_df['publication'].isin(["Dataset with its publication pending", "no publication"])]
print(len(filtered_df),'\n',filtered_df.columns)

25868 
 Index(['identifier', 'title', 'repository', 'species', 'instrument',
       'publication', 'lab_head', 'announce_date', 'keywords'],
      dtype='object')


Normalize the `publication` column by extracting the links from the html tags

In [10]:
filtered_df = filtered_df.copy()

filtered_df.loc[:, 'citing_publications_links'] = None

for i, row in filtered_df.iterrows():
    pub = str(row['publication'])  # Ensure string type
    if "href" in pub:
        match = re.findall(r'href=[\'"]([^\'"]+)[\'"]', pub)  # Extract href links
        filtered_df.at[i, 'citing_publications_links'] = match if match else None
    else:
        filtered_df.at[i, 'citing_publications_links'] = None

# Drop rows with missing links safely
filtered_df = filtered_df.dropna(subset=['citing_publications_links']).reset_index(drop=True)
filtered_df.drop(columns=['publication'], inplace=True)  # Drop original column

print(f"# n datasets with publications: {len(filtered_df)}") 
print(filtered_df.columns)

# n datasets with publications: 25696
Index(['identifier', 'title', 'repository', 'species', 'instrument',
       'lab_head', 'announce_date', 'keywords', 'citing_publications_links'],
      dtype='object')


Normalize links:


- if pubmed link, find the pmcid and the doi mapped to the pmid


- if doi link, find the doi and the pmcid mapped to the doi

In [11]:
filtered_df = filtered_df.explode('citing_publications_links', ignore_index=True)  # Split lists into rows
print(f"Length: {len(filtered_df)}")

filtered_df = extract_publication_ids_from_PX_export(filtered_df, pmid_doi_mapping, pmid_pmcid_mapping, doi_pmid_mapping, doi_pmcid_mapping)
print(f"Length should be the same: {len(filtered_df)}")

filtered_df = add_citing_publication_link_columns(filtered_df)
print(f"Length should be greater: {len(filtered_df)}")

filtered_df["citing_publications_links"] = filtered_df["citing_publications_links"].astype(str)
filtered_df["DOI"] = filtered_df["DOI"].astype(str)
filtered_df["PMCID"] = filtered_df["PMCID"].astype(str)
filtered_df["Ground_truth_source"] = "proteomexchange_search.tsv"

# Now drop duplicates
filtered_df.drop_duplicates(subset=['citing_publications_links', 'DOI', 'PMCID','identifier'], inplace=True)
print(f"Length could be a bit smaller: {len(filtered_df)}")

filtered_df[['citing_publications_links','DOI','PMID','PMCID','identifier']].sample(20)

Length: 39347
Length should be the same: 39347
Before explode sample:                           DOI        PMCID  \
0  10.1038/S41467-025-56720-1         None   
1           10.6019/PXD051312         None   
2      10.1002/prca.202400095  PMC11895760   
3      10.1002/prca.202400095  PMC11895760   
4    10.17159/SAJS.2025/18571         None   
5   10.1101/2024.04.03.587901         None   
6   10.1101/2025.02.05.636703         None   
7  10.1186/S13100-024-00339-4         None   
8  10.1186/s12964-025-02046-w  PMC11773904   
9  10.1186/s12964-025-02046-w  PMC11773904   

                           citing_publications_links  
0    [https://dx.doi.org/10.1038/S41467-025-56720-1]  
1             [https://dx.doi.org/10.6019/PXD051312]  
2  [https://dx.doi.org/10.1002/prca.202400095, ht...  
3  [https://dx.doi.org/10.1002/prca.202400095, ht...  
4      [https://dx.doi.org/10.17159/SAJS.2025/18571]  
5     [https://dx.doi.org/10.1101/2024.04.03.587901]  
6     [https://dx.doi.org/10.1101/2025

,citing_publications_links,DOI,PMID,PMCID,identifier
803,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,10.3389/fmed.2024.1406748,39219796,PMC11361967,PXD054620
55935,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,10.1038/s41467-018-06368-x,30242148,PMC6155008,PXD010870
12112,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,10.1021/ac4037679,24684310,PMC4033631,PXD000917
63576,https://dx.doi.org/10.1074/mcp.M114.047530,10.1074/mcp.M114.047530,26272980,PMC4638043,PXD002163
5982,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,10.1038/s41467-020-18494-6,33020478,PMC7536423,PXD014790
22653,https://dx.doi.org/10.1128/jvi.01624-23,10.1128/jvi.01624-23,38709105,PMC11237466,PXD050055
17301,https://dx.doi.org/10.1038/S41586-024-07963-3,10.1038/S41586-024-07963-3,None,None,PXD041214
33907,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8...,10.1186/s12864-022-08372-4,35180840,PMC8855566,PXD022499
47338,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,10.1186/s13287-020-01706-7,32434555,PMC7238576,PXD016842
7120,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,10.1038/s41467-020-15467-7,32242014,PMC7118099,PXD017464


Extraction of data from GEO

In [12]:
GEO_extract_file = "exp_input/GEO_data.csv"
GEO_df = pd.read_csv(GEO_extract_file, low_memory=False)
print(f"number of dataset ids in GEO dataframe: {len(GEO_df)}")

number of dataset ids in GEO dataframe: 248748


Extracting all the datasets with eutils api and get all the uids of GEO series datasets

In [13]:
root = ElementTree.fromstring(requests.get("https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi", 
                                           params={"db": "gds", "term": '"gse"[Entry Type]', "retmax": "1000000",  "retmode": "xml"}
                                           ).content)

gse_ids = [id_elem.text for id_elem in root.findall(".//Id")]
missing_ids = set(gse_ids)-set([str(x) for x in GEO_df.uid])

print(f"Total datasets found: {len(gse_ids)}")
print("Missing GSE IDs:", len(missing_ids))  # Show difference with data we already got

Total datasets found: 249060
Missing GSE IDs: 326


Add new datasets from GEO to the GEO dataframe

In [14]:
add_new_data = False # add new datasets to the GEO_df

if add_new_data:
    new_GEO_series_data = {}
    i, mxm = 0, 1
    while True:
        print(f"Progress: {min(i / len(missing_ids), 1) * 100} %")
        if i > len(missing_ids):
            break
        try:
            new_data = fetch_GEO_data(list(missing_ids),"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi",i,mxm)
            for uid, details in new_data["result"].items():
                if uid == "uids":  # Ignore metadata key
                    continue
                new_GEO_series_data[uid] = details
        except Exception as e:
            print(f"Error at {i}, {mxm} with error: {e}")
                
        i += 1
        mxm += 1
        time.sleep(0.005) 

    GEO_df = pd.concat([GEO_df, pd.DataFrame(new_GEO_series_data).T], axis=0)

We will update the filtered_df with the new datasets from GEO, after matching the schema

Add the url to the GEO datasets by reconstruction from pubmedids

In [15]:
# eval string to list
GEO_df['pubmedids'] = GEO_df['pubmedids'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
print(len(GEO_df))
GEO_df = GEO_df[GEO_df['pubmedids'].astype(bool)]  # Drop rows with empty lists
print(len(GEO_df))
GEO_df = GEO_df.explode('pubmedids', ignore_index=True)  # Split lists into rows
print(len(GEO_df))

248748
194093
208151


- if pubmed link, find the pmcid and the doi mapped to the pmid


- Create Links for the publications

In [16]:
GEO_df['PMCID'] = GEO_df['pubmedids'].map(
    lambda x: pmid_pmcid_mapping.get(x, None) if isinstance(x, str) else None )
GEO_df['DOI'] = GEO_df['pubmedids'].map(
    lambda x: pmid_doi_mapping.get(x, None) if isinstance(x, str) else None)
GEO_df[['pubmedids','PMCID','DOI']].sample(10)

,pubmedids,PMCID,DOI
50904,36443881,PMC9702864,10.1186/s12985-022-01930-y
40546,36651277,PMC9943675,10.1093/nar/gkac1237
72891,34019588,PMC8174705,10.1371/journal.ppat.1009577
121809,29329517,PMC5767043,10.1186/s12864-017-4367-1
126091,22955616,PMC3439153,10.1038/nature11247
155098,28739826,PMC5607407,10.1128/IAI.00389-17
77470,39614284,PMC11606223,10.1186/s13062-024-00556-0
15949,22955616,PMC3439153,10.1038/nature11247
180855,22406535,PMC3314474,10.1172/JCI61392
87931,35967340,PMC9366357,10.3389/fimmu.2022.960918


In [17]:
# Generate citing_publications_links as a list of links
GEO_df = add_citing_publication_link_columns(GEO_df)

GEO_df[['pubmedids','PMCID','DOI','citing_publications_links']].sample(10)

Before explode sample:                        DOI        PMCID  \
0                     None         None   
1   10.1093/immhor/vlae005  PMC11841973   
2      10.1093/nar/gkaf109  PMC11840560   
3                     None         None   
4                     None         None   
5                     None         None   
6                     None         None   
7                     None         None   
8                     None         None   
9  10.1073/pnas.2416384122  PMC11892594   

                           citing_publications_links  
0                                                 []  
1  [https://dx.doi.org/10.1093/immhor/vlae005, ht...  
2  [https://dx.doi.org/10.1093/nar/gkaf109, https...  
3                                                 []  
4                                                 []  
5                                                 []  
6                                                 []  
7                                                 []  
8       

,pubmedids,PMCID,DOI,citing_publications_links
302912,25315157,PMC4312125,10.1002/cam4.351,https://dx.doi.org/10.1002/cam4.351
332433,23177624,PMC3625945,10.1016/j.celrep.2012.10.017,https://dx.doi.org/10.1016/j.celrep.2012.10.017
36017,38426156,PMC10902752,10.3892/ol.2024.14291,https://dx.doi.org/10.3892/ol.2024.14291
264477,27077850,PMC4848955,10.3390/ijms17040499,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...
240296,28704530,PMC5509291,10.1371/journal.pone.0181155,https://dx.doi.org/10.1371/journal.pone.0181155
376347,17724126,PMC2118696,10.1084/jem.20070041,https://dx.doi.org/10.1084/jem.20070041
218211,30360761,PMC6203201,10.1186/s13059-018-1551-9,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...
226530,29638215,PMC5896888,10.7554/eLife.32472,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...
123048,34591327,PMC9812230,10.1096/fj.202100862R,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9...
21182,38746145,PMC11092767,10.1101/2024.05.05.592422,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...


In [18]:
# drop NaN citing_publications_links
GEO_df = GEO_df.dropna(subset=['citing_publications_links'])  # Remove NaNs
print(len(GEO_df))

352982


Union of the two dataframes

In [19]:
print(f"len filtered_df: {len(filtered_df)}")
print(f"len GEO_df: {len(GEO_df)}")

len filtered_df: 44281
len GEO_df: 352982


In [20]:
GEO_df['Ground_truth_source'] = 'GEO_API'  # Add source column

In [21]:
column_mappings = { # Manually map columns
    'accession': 'identifier',
    'taxon': 'species',
    'repository': 'entrytype',
    'publication_date': 'pdat',
    'pubmedids': 'PMID',
}

In [22]:
GEO_df = GEO_df.rename(columns=column_mappings)  # Rename columns
GEO_df['repository'] = 'GEO'  # Add repository column
filtered_df = pd.concat([filtered_df, GEO_df], axis=0, ignore_index=True)  # Concatenate dataframes

In [23]:
filtered_df.iloc[::10000].head(10)  # Show every 10000th row

,identifier,title,repository,species,instrument,lab_head,announce_date,keywords,citing_publications_links,PMID,...,pdat,suppfile,samples,relations,extrelations,n_samples,projects,ftplink,geo2r,bioproject
0,PXD059466,Endothelial SHANK3 Regulates Tight Junctions i...,PRIDE,Mus musculus,Orbitrap Fusion Lumos,Il Hwan Kim,2025-02-07,"BioID2, SHANK3, bEnd3. Cells",https://dx.doi.org/10.1038/S41467-025-56720-1,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,PXD000235,bHLH Transcription Factors that Facilitate K+ ...,PRIDE,Arabidopsis thaliana (Mouse-ear cress),LTQ,Y Takahashi,2024-10-07,"LTQ, stomatal guard cells",https://dx.doi.org/10.6019/PXD000235,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20000,PXD038874,Sulconazole induces PANoptosis to increase rad...,iProX,Homo sapiens,Orbitrap Fusion ETD,Liyan Xu,2022-12-17,"PANoptosis, Sulconazole, oxidative stress",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,37076047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30000,PXD014403,Degradation of Quaternary amine in methanogen,MassIVE,Methanolobus vulcani,LTQ Orbitrap XL,"Donald J. Ferguson, XIN WANG",2021-03-29,"methanogen, quaternary amine",https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,31787957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40000,PXD004559,"Mapping and quantification of over 2,000 O-lin...",PRIDE,Homo sapiens,LTQ Orbitrap Elite,Christina Woo,2018-01-25,"O-GlcNAc, O-linked N-acetylglucosamine, T cell...",https://dx.doi.org/10.1074/mcp.RA117.000261,29351928,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50000,GSE270865,Genomic and Histologic Analysis of Uterine Lei...,GEO,Homo sapiens,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,39691991,...,2024/11/05,"MTX, TSV","[{'accession': 'GSM8353848', 'title': 'STSW3_S...",[],[],4.0,[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE270nn...,no,PRJNA1128661
60000,GSE253632,Chromatin conformation and histone modificatio...,GEO,Homo sapiens,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,39025878,...,2024/06/20,HIC,"[{'accession': 'GSM8024403', 'title': 'Human k...",[],[],6.0,[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE253nn...,no,PRJNA1066382
70000,GSE259050,Control ChIP-seq from whole organism (ENCSR821...,GEO,Caenorhabditis elegans,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,22955616,...,2024/02/27,TXT,"[{'accession': 'GSM8111018', 'title': 'Control...",[],[],1.0,"[{'name': 'modENCODE', 'url': 'https://www.ncb...",ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE259nn...,no,PRJNA63463
80000,GSE238202,CLIP-seq analysis of hnRNPA1-RNA interactions,GEO,Homo sapiens,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,38985864,...,2023/12/31,TXT,"[{'accession': 'GSM7660285', 'title': 'WT_U_re...",[],[],6.0,[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE238nn...,no,PRJNA998366
90000,GSE206190,Profiling of RNA-binding protein LSM14B target...,GEO,Mus musculus,NaN,NaN,NaN,NaN,https://dx.doi.org/10.1002/advs.202300043,37083226,...,2023/09/08,BIGWIG,"[{'accession': 'GSM6245835', 'title': 'LM-2000...",[],[],2.0,[],ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE206nn...,no,PRJNA849588


In [24]:
filtered_df.to_parquet('exp_input/Table_datasets.parquet', index=False)

## Creating Publications Data Table

We start from ProteomeCentral and GEO table we created before

In [25]:
filtered_df = pd.read_parquet('exp_input/Table_datasets.parquet')

In [26]:
print(len(filtered_df))
filtered_df.head(3)

397263


,identifier,title,repository,species,instrument,lab_head,announce_date,keywords,citing_publications_links,PMID,...,pdat,suppfile,samples,relations,extrelations,n_samples,projects,ftplink,geo2r,bioproject
0,PXD059466,Endothelial SHANK3 Regulates Tight Junctions i...,PRIDE,Mus musculus,Orbitrap Fusion Lumos,Il Hwan Kim,2025-02-07,"BioID2, SHANK3, bEnd3. Cells",https://dx.doi.org/10.1038/S41467-025-56720-1,None,...,None,None,None,None,None,NaN,None,None,None,None
1,PXD051312,Systemic changes in early pregnancy in the mar...,PRIDE,Equus caballus,"Exactive Plus, Orbitrap Exploris 480",Dr Aleona,2025-02-07,"MRP, TTR, biomarker, early pregnancy loss, sec...",https://dx.doi.org/10.6019/PXD051312,None,...,None,None,None,None,None,NaN,None,None,None,None
2,PXD051312,Systemic changes in early pregnancy in the mar...,PRIDE,Equus caballus,"Exactive Plus, Orbitrap Exploris 480",Dr Aleona,2025-02-07,"MRP, TTR, biomarker, early pregnancy loss, sec...",https://dx.doi.org/10.1002/prca.202400095,39912552,...,None,None,None,None,None,NaN,None,None,None,None


Some statistics on our publications data

look at the publications one by one

In [27]:
filtered_df = filtered_df.explode('citing_publications_links')  # Split lists into rows

In [28]:
df_publications = filtered_df[['citing_publications_links','DOI','PMID','PMCID']]

df_publications = df_publications.sort_values(by='citing_publications_links', key=lambda x: x.str.contains('doi', na=False), ascending=False)

df_publications = df_publications.drop_duplicates(subset=['PMCID', 'DOI'], keep='first')

len(df_publications)

126461

In [29]:
print(len(df_publications))
print(df_publications.dtypes)
df_publications.head(3)

126461
citing_publications_links    object
DOI                          object
PMID                         object
PMCID                        object
dtype: object


,citing_publications_links,DOI,PMID,PMCID
0,https://dx.doi.org/10.1038/S41467-025-56720-1,10.1038/S41467-025-56720-1,None,None
220686,https://dx.doi.org/10.1074/jbc.RA119.011506,10.1074/jbc.RA119.011506,31753917,PMC6937554
220626,https://dx.doi.org/10.1084/jem.20190580,10.1084/jem.20190580,31816633,PMC7062527


In [30]:
df_publications.astype(str).to_parquet('exp_input/Table_publications.parquet', index=False)

## Create the ground truth table

each row is a record of a dataset citation in a doi

In [31]:
import pandas as pd

In [32]:
Publications_table = pd.read_parquet('exp_input/Table_publications.parquet')
Datasets_table = pd.read_parquet('exp_input/Table_datasets.parquet')

We iterate over all the datasets and use their publication urls to find the corresponding dois. Since not all papers had dois, we will create a Publication Identifier table

In [33]:
Datasets_table.columns

Index(['identifier', 'title', 'repository', 'species', 'instrument',
       'lab_head', 'announce_date', 'keywords', 'citing_publications_links',
       'PMID', 'PMCID', 'DOI', 'Ground_truth_source', 'Unnamed: 0', 'uid',
       'gds', 'summary', 'gpl', 'gse', 'entrytype', 'gdstype', 'pdat',
       'suppfile', 'samples', 'relations', 'extrelations', 'n_samples',
       'projects', 'ftplink', 'geo2r', 'bioproject'],
      dtype='object')

In [34]:
Publication_Dataset_Citations_Ground_Truth = Datasets_table[['identifier','repository','citing_publications_links','Ground_truth_source']].rename({'citing_publications_links':'citing_publication_link','Ground_truth_source' : 'citation_record_source'}, axis=1)

In [35]:
# create a column with the indicator if the dataset citation record should be found on a PMC vs DOI, extrct that from link
Publication_Dataset_Citations_Ground_Truth['citation_record_from_doi'] = Publication_Dataset_Citations_Ground_Truth['citing_publication_link'].apply(lambda x: 1 if 'doi' in x else 0)

In [36]:
Publication_Dataset_Citations_Ground_Truth.to_parquet('exp_input/dataset_citation_records_Table.parquet', index=False)